<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Download-libraries-for-Google-Colab-and-Download-Embeddings" data-toc-modified-id="Download-libraries-for-Google-Colab-and-Download-Embeddings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download libraries for Google Colab and Download Embeddings</a></span></li><li><span><a href="#Import-Data" data-toc-modified-id="Import-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Data</a></span></li><li><span><a href="#Load-Embedding-and-Create-Embedding-Layer" data-toc-modified-id="Load-Embedding-and-Create-Embedding-Layer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Embedding and Create Embedding Layer</a></span></li><li><span><a href="#Define-and-Train-Network" data-toc-modified-id="Define-and-Train-Network-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Define and Train Network</a></span></li><li><span><a href="#Pass-Train-and-Test-Data-Through-Network-and-Save-for-Stacking" data-toc-modified-id="Pass-Train-and-Test-Data-Through-Network-and-Save-for-Stacking-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Pass Train and Test Data Through Network and Save for Stacking</a></span></li></ul></div>

# Download libraries for Google Colab and Download Embeddings

In [0]:
! pip install pandas==0.23.4

    100% |████████████████████████████████| 8.9MB 3.9MB/s 
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
      Successfully uninstalled pandas-0.22.0


In [0]:
import pandas as pd
pd.__version__

'0.22.0'

In [0]:
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2018-12-17 20:35:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-12-17 20:35:40--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  38.4MB/s    in 96s     

2018-12-17 20:37:17 (8.52 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [0]:
! unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


# Import Data

In [0]:
import numpy as np
import pandas as pd
import csv

In [0]:
pd.__version__

'0.23.4'

In [0]:
df_train = pd.read_csv("train_data.csv",doublequote=True,quotechar='"',sep=",").drop("is_duplicate",axis=1)

In [0]:
df_t_labels= pd.read_csv('train_labels.csv', encoding='utf-8')
df_t_labels.head()

,id,is_duplicate
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [0]:
df_train = df_train.merge(df_t_labels,on=["id"],how="inner")
df_train.head()
df_train['id'] = df_train['id'].apply(str)
df_train['is_duplicate'] = df_train['is_duplicate'].apply(int)

In [0]:
df_train['question1'].fillna('', inplace=True)
df_train['question2'].fillna('', inplace=True)

In [0]:
pd.__version__

'0.23.4'

In [0]:
df_test = pd.read_csv('test_data.csv',doublequote=True)
df_test['test_id'] = df_test['test_id'].apply(str)

In [0]:
df_all = pd.concat((df_train, df_test))
df_all['question1'].fillna('', inplace=True)
df_all['question2'].fillna('', inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import itertools

In [0]:
counts_vectorizer = CountVectorizer(max_features=10000-1).fit(
    itertools.chain(df_all['question1'], df_all['question2']))

other_index = len(counts_vectorizer.vocabulary_)

In [0]:
import re
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import keras 
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [0]:
words_tokenizer = re.compile(counts_vectorizer.token_pattern)

In [0]:
def create_padded_seqs(texts, max_len=30):
    seqs = texts.apply(lambda s: 
        [counts_vectorizer.vocabulary_[w] if w in counts_vectorizer.vocabulary_ else other_index
         for w in words_tokenizer.findall(s.lower())])
    return pad_sequences(seqs, maxlen=max_len)

In [0]:
X1_train, X1_val, X2_train, X2_val, y_train, y_val, train_id, train_val_id = \
    train_test_split(create_padded_seqs(df_train['question1']),
                     create_padded_seqs(df_train['question2']),
                     df_train['is_duplicate'].values,
                     df_train.id,
                     stratify=df_train['is_duplicate'].values,
                     test_size=0.3, random_state=2018)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [0]:
train_id_pd = pd.DataFrame(train_id.values)
train_val_id_pd = pd.DataFrame(train_val_id.values)

In [0]:
train_id_pd.to_csv("train_id_merge_manhat.csv",index=False)
train_val_id_pd.to_csv("train_val_id_merge_manhat.csv", index=False)

# Load Embedding and Create Embedding Layer

In [0]:
import os
import numpy as np

GLOVE_DIR = ""

embeddings_index = {}
line_num = 0

f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'))
ff = list(f)
f.close()

for line in ff:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    line_num += 1
    if line_num == 400000:
        break;

In [0]:
EMBEDDING_DIM = 300

word_index = counts_vectorizer.vocabulary_

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding

MAX_SEQUENCE_LENGTH = X1_train.shape[1]

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

# Define and Train Network

In [0]:
# Got the inspiration from
# http://www.mit.edu/~jonasm/info/MuellerThyagarajan_AAAI16.pdf

In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
from keras.layers import *
from keras.models import Model

In [0]:
#Manhatten LSTM Function
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [0]:
manhat_merge = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))

In [0]:
words_embedding_layer = embedding_layer
seq_embedding_layer = LSTM(256, activation='tanh')

input1_tensor = Input(X1_train.shape[1:])
input2_tensor = Input(X2_train.shape[1:])

words_embedding_layer1 = words_embedding_layer(input1_tensor)
words_embedding_layer2 = words_embedding_layer(input2_tensor)

seq_embedding_layer1 = seq_embedding_layer(words_embedding_layer1) 
seq_embedding_layer2 = seq_embedding_layer(words_embedding_layer2)

manhat_distance = manhat_merge([seq_embedding_layer1, seq_embedding_layer2])

ouput_layer = Dense(1, activation='sigmoid')(manhat_distance)

model = Model([input1_tensor, input2_tensor], [ouput_layer])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 300)      3000000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 256)          570368      embedding_1[0][0]                
          

In [0]:
! mkdir gloverLSTM256.merge.manhat

In [0]:
from keras.callbacks import ModelCheckpoint  


directory = "gloverLSTM256.merge.manhat/dropout.{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5"

checkpointer = ModelCheckpoint(filepath=directory, 
                               verbose=1, save_best_only=False,monitor="val_loss")


In [0]:
model.fit([X1_train, X2_train], 
          y_train, 
            batch_size=128, 
            epochs=4,
            callbacks=[checkpointer],
            validation_data=([X1_val, X2_val], y_val))

Train on 226214 samples, validate on 96950 samples
Epoch 1/4
226214/226214 [==============================] - 315s 1ms/step - loss: 0.5380 - acc: 0.7456 - val_loss: 0.5755 - val_acc: 0.7129

Epoch 00001: saving model to gloverLSTM256.merge.manhat/dropout.01-0.54-0.58.hdf5
Epoch 2/4
226214/226214 [==============================] - 314s 1ms/step - loss: 0.5278 - acc: 0.7544 - val_loss: 0.5760 - val_acc: 0.7112

Epoch 00002: saving model to gloverLSTM256.merge.manhat/dropout.02-0.53-0.58.hdf5
Epoch 3/4
218112/226214 [===========================>..] - ETA: 9s - loss: 0.5183 - acc: 0.7624

KeyboardInterrupt: ignored

In [0]:
Train on 226214 samples, validate on 96950 samples
Epoch 1/8
226214/226214 [==============================] - 317s 1ms/step - loss: 0.6620 - acc: 0.6312 - val_loss: 0.6572 - val_acc: 0.6312

Epoch 00001: saving model to gloverLSTM256.merge.manhat/dropout.01-0.66-0.66.hdf5
Epoch 2/8
226214/226214 [==============================] - 315s 1ms/step - loss: 0.6569 - acc: 0.6312 - val_loss: 0.6558 - val_acc: 0.6312

Epoch 00002: saving model to gloverLSTM256.merge.manhat/dropout.02-0.66-0.66.hdf5
Epoch 3/8
226214/226214 [==============================] - 313s 1ms/step - loss: 0.6286 - acc: 0.6312 - val_loss: 0.6116 - val_acc: 0.6312

Epoch 00003: saving model to gloverLSTM256.merge.manhat/dropout.03-0.63-0.61.hdf5
Epoch 4/8
226214/226214 [==============================] - 314s 1ms/step - loss: 0.5992 - acc: 0.6748 - val_loss: 0.5978 - val_acc: 0.6817

Epoch 00004: saving model to gloverLSTM256.merge.manhat/dropout.04-0.60-0.60.hdf5
Epoch 5/8
226214/226214 [==============================] - 314s 1ms/step - loss: 0.5830 - acc: 0.6989 - val_loss: 0.5885 - val_acc: 0.6936

Epoch 00005: saving model to gloverLSTM256.merge.manhat/dropout.05-0.58-0.59.hdf5
Epoch 6/8
226214/226214 [==============================] - 314s 1ms/step - loss: 0.5702 - acc: 0.7138 - val_loss: 0.5828 - val_acc: 0.7005

Epoch 00006: saving model to gloverLSTM256.merge.manhat/dropout.06-0.57-0.58.hdf5
Epoch 7/8
226214/226214 [==============================] - 314s 1ms/step - loss: 0.5593 - acc: 0.7251 - val_loss: 0.5799 - val_acc: 0.7065

Epoch 00007: saving model to gloverLSTM256.merge.manhat/dropout.07-0.56-0.58.hdf5
Epoch 8/8
226214/226214 [==============================] - 313s 1ms/step - loss: 0.5487 - acc: 0.7354 - val_loss: 0.5792 - val_acc: 0.7028

Epoch 00008: saving model to gloverLSTM256.merge.manhat/dropout.08-0.55-0.58.hdf5
<keras.callbacks.History at 0x7fc2bd549b38>

In [0]:
#Train on 226214 samples, validate on 96950 samples
#Epoch 1/8
#226214/226214 [==============================] - 314s 1ms/step - loss: 0.6167 - acc: 0.7070 - val_loss: 0.5883 - val_acc: 0.7129
#
#Epoch 00001: saving model to gloverLSTM256.merge.manhat/dropout.01-0.62-0.59.hdf5
#Epoch 2/8
#226214/226214 [==============================] - 335s 1ms/step - loss: 0.5680 - acc: 0.7235 - val_loss: 0.5589 - val_acc: 0.7221
#
#Epoch 00002: saving model to gloverLSTM256.merge.manhat/dropout.02-0.57-0.56.hdf5
#Epoch 3/8
#226214/226214 [==============================] - 318s 1ms/step - loss: 0.5400 - acc: 0.7337 - val_loss: 0.5381 - val_acc: 0.7287
#
#Epoch 00003: saving model to gloverLSTM256.merge.manhat/dropout.03-0.54-0.54.hdf5
#Epoch 4/8
#226214/226214 [==============================] - 319s 1ms/step - loss: 0.5198 - acc: 0.7441 - val_loss: 0.5250 - val_acc: 0.7384
#
#Epoch 00004: saving model to gloverLSTM256.merge.manhat/dropout.04-0.52-0.53.hdf5
#Epoch 5/8
#226214/226214 [==============================] - 323s 1ms/step - loss: 0.5044 - acc: 0.7546 - val_loss: 0.5099 - val_acc: 0.7559
#
#Epoch 00005: saving model to gloverLSTM256.merge.manhat/dropout.05-0.50-0.51.hdf5
#Epoch 6/8
#226214/226214 [==============================] - 351s 2ms/step - loss: 0.4915 - acc: 0.7655 - val_loss: 0.5068 - val_acc: 0.7589
#
#Epoch 00006: saving model to gloverLSTM256.merge.manhat/dropout.06-0.49-0.51.hdf5
#Epoch 7/8
#226214/226214 [==============================] - 382s 2ms/step - loss: 0.4809 - acc: 0.7753 - val_loss: 0.5001 - val_acc: 0.7690
#
#Epoch 00007: saving model to gloverLSTM256.merge.manhat/dropout.07-0.48-0.50.hdf5
#Epoch 8/8
#226214/226214 [==============================] - 332s 1ms/step - loss: 0.4708 - acc: 0.7852 - val_loss: 0.4975 - val_acc: 0.7697
#
#Epoch 00008: saving model to gloverLSTM256.merge.manhat/dropout.08-0.47-0.50.hdf5
#<keras.callbacks.History at 0x7f160f657630>

In [0]:
# Load Best Epoch
model.load_weights("gloverLSTM256.merge.manhat/dropout.04-0.60-0.60.hdf5")

# Pass Train and Test Data Through Network and Save for Stacking

In [0]:
features_model_16 = Model([input1_tensor, input2_tensor], manhat_distance)
features_model_16.compile(loss='mse', optimizer='adam')

In [0]:
train_preds = [create_padded_seqs(df_train['question1']),create_padded_seqs(df_train['question2'])]
test_preds = [create_padded_seqs(df_test['question1']),create_padded_seqs(df_test['question2'])]

In [0]:
F_train_16 = features_model_16.predict(train_preds, batch_size=128)
F_test_16 = features_model_16.predict(test_preds, batch_size=128)

In [0]:
F_train_1 = model.predict(train_preds, batch_size=128)
F_test_1 = model.predict(test_preds, batch_size=128)

In [0]:
test_1 = pd.concat([df_test.test_id,pd.DataFrame(F_test_1)],axis=1)
test_16 = pd.concat([df_test.test_id,pd.DataFrame(F_test_16)],axis=1)

In [0]:
print(test_1.shape)
print(df_test.shape)
test_16.head(1)

(81126, 2)
(81126, 3)


,test_id,0
0,15,3.543442e-30


In [0]:
train_1 = pd.concat([df_train.id,pd.DataFrame(F_train_1)],axis=1)
train_16 = pd.concat([df_train.id,pd.DataFrame(F_train_16)],axis=1)

In [0]:
print(train_1.shape)
print(df_train.shape)
train_16.head(1)

(323164, 2)
(323164, 4)


,id,0
0,0,0.887289


In [0]:
train_16.to_csv("train_merge_manhat.csv",index=False)
train_1.to_csv("train_output_merge_manhat.csv",index=False)

In [0]:
pd.read_csv("train_merge_manhat.csv").head()

,id,0
0,0,0.887289
1,1,0.796232
2,2,0.222969
3,3,0.724733
4,4,0.666681


In [0]:
test_16.to_csv("test_merge_manhat.csv",index=False)
test_1.to_csv("test_output_merge_manhat.csv",index=False)

In [0]:
pd.read_csv("train_1.csv").head()

,id,0
0,0,0.654116
1,1,0.020497
2,2,0.078235
3,3,0.001608
4,4,0.007873
